In [1]:
import pandas as pd
import networkx as nx
import itertools

In [2]:
df = pd.read_json("rawfilteredbydate.json")

In [3]:
df.head()

,caption,date
0,The Perlman Music Program Class of 2014,2014-08-22
1,Toby Perlman addresses the crowd,2014-08-22
2,Itzhak Perlman and Catherine Cho perform with...,2014-08-22
3,"Julian Beck, Lyn Barlow, Ted Ayala, Elinor He...",2014-08-22
4,Toby Perlman and guests,2014-08-22


In [4]:
df["caption"] = df['caption'].apply(lambda x: x.replace("\n",""))

In [5]:
df = df[df['caption'].apply(lambda x: len(x)<=250)]

In [58]:
restr = ["The", "Dr", "Verdana", "Helvetica" ,"Chairman","Director",
         "Guests","President","Hospital","Special Surgery","Executive Director"
         ,"New York","Museum","Photographs","Children","The Society","Dinner",
        "Board Member","Castle","President and CEO","Directors","Clockwise","Doubles","BFAnyc","Standing"]

In [59]:
def replaceWord(s):
    for i in range(len(restr)):
        s = s.replace(restr[i],"")
    s = s.replace("and",",")
    return s

In [60]:
proper_noun_regex = r'([A-Z][A-Za-z-]*(,?\s+(\b(and|von|de|van der|van|der)\b))?(\s[A-Z][A-Za-z\'-]*)*)'

In [61]:
import re

In [62]:
G=nx.MultiGraph()
for caption in df.caption:
    name = []
    caption_temp  = re.findall(proper_noun_regex,caption)
    for j in range(len(caption_temp)):
        name_temp = caption_temp[j]
        if len(name_temp[0])>5:
            if name_temp[0] == "PatrickMcMullan":
                name.append("Patrick McMullan")
            elif name_temp[0] == "AnnieWatt":
                name.append("Annie Watt")
            elif "Mayor" in name_temp[0]:
                temp = name_temp[0].replace("Mayor ","")
                name.append(temp)
            else:
                name.append(name_temp[0])
        for word in restr:
            if word in name:
                name.remove(word)
    if len(name) > 2:
        _=[G.add_edge(c[0],c[1],weight=1) for c in itertools.combinations(name,2)]
    elif len(name) > 1:
        G.add_edge(name[0],name[1],weight=1)        

In [63]:
Gg = nx.Graph()
for u,v,data in G.edges_iter(data=True):
    if Gg.has_edge(u,v):
        Gg[u][v]['weight'] += 1
    else:
        Gg.add_edge(u, v, weight=1)

## degrees

In [64]:
degrees=pd.DataFrame(G.degree().values(),index=G.degree().keys()).sort(0,ascending=False)

In [65]:
degrees = degrees.reset_index()

In [66]:
degrees.columns = ["name","number"]

In [57]:
degrees.name[80:100]

80             Susan Shin
81          Barbara Regna
82               Liz Peek
83            John Demsey
84    Dawne Marie Grannum
85    Alexandra Lind Rose
86         Consul General
87        Deborah Roberts
88        Nathalie Kaplan
89       Alexia Hamm Ryan
90           Nina Griscom
91            Fern Mallis
92           Bette Midler
93           Lisa Anastos
94               Standing
95         Patricia Shiah
96                 John J
97        Barbara Walters
98     Beth Rudin DeWoody
99         Julie Skarratt
Name: name, dtype: object

In [54]:
[i for i in degrees.name if "Kanavos" in i]

[u'Kanavos',
 u'Dayssi Kanavos',
 u'Paul Kanavos',
 u'Kanavos, and Madeline Hult Elghanayan',
 u'Kanavos, and Claudia Lebenthal',
 u'Peter Kanavos',
 u'Sophia Kanavos',
 u'Kanavos, and Donya Bommer',
 u'Kanavos and Marisa Brown',
 u'Kanavos, and Nicholas Scoppetta',
 u'Kanavos, and Amy McFarland',
 u"Kanavos, and Sha'Rhone Frost",
 u'Olarte de Kanavos',
 u'Kanavos and Dayssi Olarte',
 u'Kanavos and Daniel OlarteCarolina Alvarez Mathies',
 u'Kanavos, and Nacho Figueras',
 u'Kanavos, and Cherry',
 u'Kanavos, and Mary Brown',
 u'Kanavos and Ashley McDermott',
 u'Kanavos and Valesca Guerrand-Hermes',
 u'Kanavos, and Natalia Echavarria',
 u'Kanavos and Sophia Kanavos',
 u'Kanavos, and Norma Olarte Becker',
 u'Daisy Kanavos',
 u'Kanavos and Daniel Olarte',
 u'Kanavos and Andrea Greeven Douzet',
 u'Kanavos and Brian Reyes',
 u'Dayssie Olarte DeKanavos',
 u'Kanavos and Soigne Kothari',
 u'Kanavos and Angela Barco',
 u'Kanavos and Juan Carlos Arcila Duque',
 u'Kanavos and Mark Gilbertson',
 u'K

In [39]:
degreeres = map(lambda x,y: (x,y),degrees.name[0:100], degrees.number[0:100])

In [40]:
import pickle

In [41]:
with open('degreeres.pickle', 'wb') as f:
    pickle.dump(degreeres, f)

In [14]:
pr=nx.pagerank(Gg)
pranalysis=pd.DataFrame(pr.values(),index=pr.keys()).sort(0,ascending=False)